In [11]:
import yaml

config = """

input:
        
    # Define the input image with the pixel data.
    # These data come with a bad pixel mask and a weight map.
    image_file_name: DESJ233551.8640-515217.7600_z.fits
    image_hdu: 1
    badpix_hdu: 2
    
    # Note: The weight map is expeected to be an inverse variance map, containing only the non-signal
    #       part of the noise (sky, readnoise, dark current, etc.).  This is what DES provides.
    #       If you instead have a variance map (e.g. from LSST), you can tell Piff to invert it using
    #           invert_weight: True
    #       And if it includes Poisson noise from the signal (e.g. also LSST), Piff can remove it with
    #           remove_signal_from_weight: True
    weight_hdu: 3
        
    # Define the input catalog.
    cat_file_name: DESJ233551.8640-515217.7600_z.fits
        
    # Either (x,y) or (ra,dec) are required columns saying where the objects are.
    x_col: XWIN_IMAGE
    y_col: YWIN_IMAGE
        
    # If flags is set, it will ignore flagged rows.
    flag_col: FLAGS
        
    # You can optionally specify any other columns you want to read.
    # These will be saved as additional properties of each star, which may be used in various ways.
    property_cols: [SPREAD_MODEL]
        
    # This sets how large a stamp to pull out of the full image for each input object.
    stamp_size: 25


select:
    # The default is to use all input objects as candidate stars.
    # Here we refine that a bit by selecting rows based on the SExtractor "SPREAD_MODEL" value.
    # SPREAD_MODEL is a measure of how much bigger an object is than the SExtractor software's estimate
    # of the PSF size, so objects with SPREAD_MODEL close to zero are likely stars.
    type: Properties
        
    # This can be any eval-able string.
    # As mentioned, here we use the SPREAD_MODEL property we read in from the input catalog.
    where: "np.abs(SPREAD_MODEL) < 3.e-4"
        
    # Mark 20% of stars as "reserve" stars.  Don't use them for fitting.                     
    # In machine learning lingo, this would be a validation set.
    reserve_frac: 0.2    
        
    # Specifying a seed makes the choice of reserve stars deterministic.  
    # Typically set on the command line, since you'd want this to be different for each image.
    seed: 1234   
        
    # Remove >3 sigma outlier in the HSM-measured size.
    # HSM (Hirata, Seljak and Mandelbaum) is an adaptive moment algorithm for measuring size and shapes, 
    # which is pretty stable and is included with GalSim.
    hsm_size_reject: 3   
        
    # Remove objects with estimated S/N < 50
    min_snr: 50          


psf:
    # The default "type" is "Simple", which just has a single model and interpolation scheme.
        
    model:
        # This model uses a gridded array of pixels, each with a free amplitude.
        # Here, it is a 21x21 grid of 0.27 arcsec pixels.
        type: PixelGrid
        scale: 0.27
        size: 21

    interp:
        # There are two different polynomial classes one can use for interpolation.
        # For models that can delay the final calculation of the parameters (which PixelGrid can),
        # the more complicated BasisPolynomial should normally be preferred.  It lets the model type
        # define the effect of the parameters, but the actual fit is delayed to use all the stars at
        # once combining the fit with the interpolation coefficients.  
        # This generally works better than completing the fit at each star and then interpolating,
        # but not all models can work in this mode.
        type: BasisPolynomial
        order: 3
            
    outliers:
        # After the 2nd and subsequent iterations, look for outliers whose fits have a bad chisq value.
        # This is really defined in terms of a chisq probability distribution falling below some value,
        # but for convenience we let that probability be set in terms of "nsigma" using the 1-d Gaussian
        # probability distribution meaning for n sigma.
        type: Chisq
        nsigma: 4

output:
    # For now, just output the file with the PSF solution.  We'll add more to this section below.
    dir: output
    file_name: DESJ233551.8640-515217.7600_z.piff
"""

config = yaml.safe_load(config)

In [12]:
import piff

piff.piffify(config)

Getting wcs from image file DESJ233551.8640-515217.7600_z.fits
Reading image file DESJ233551.8640-515217.7600_z.fits


OSError: Expecting at least 4 HDUs in galsim.read